This notebook will run a live demo on Jetson Nano using [JetCam](https://github.com/NVIDIA-AI-IOT/jetcam) to acquire images from the camera.  First,
let's start the camera.  See the JetCam examples for details.

In [ ]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224)

camera.running = True

Now, let's connect the camera's value to a widget to display.

In [ ]:
from jetcam.utils import bgr8_to_jpeg
import traitlets
import ipywidgets

image_w = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_w, 'value'), transform=bgr8_to_jpeg)

display(image_w)

Next, we'll load the TensorRT model.  (We assume you followed the conversion notebook and saved to the path ``resnet18_trt.pth``)

In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('resnet18_trt.pth'))

The following function will be used to pre-process images from the camera

In [ ]:
import cv2
import numpy as np
import torchvision

device = torch.device('cuda')
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

This text area will be used to display the class predictions.

In [ ]:
text = ipywidgets.Textarea()
display(text)

We load the imagenet labels to associate the neural network output with a class name.

In [ ]:
import json

with open('imagenet_labels.json', 'r') as f:
    labels = json.load(f)

Finally, we create our execution function, which we attach as a callback to the camera's ``value`` attribute.

Whenever the camera's value is updated (which it will be for each frame, since we set ``camera.running = True``).  This function will be called
describing how the value changed.  The new camera value will be stored in ``change['new']``.

In [ ]:
def execute(change):
    image = change['new']
    output = model_trt(preprocess(image).half()).detach().cpu().numpy().flatten()
    idx = output.argmax()
    text.value = labels[idx]

camera.observe(execute, names='value')